In [ ]:
import pandas as pd
import numpy as np
from tkinter import filedialog
import tkinter as tk
import openpyxl
import customtkinter as ctk
from customtkinter import filedialog


### Subir el archivo y guardarlo en un dataframe

In [ ]:
root = ctk.CTk()
root.filename = filedialog.askopenfilename(initialdir="/", title="Selecciona un archivo", filetypes=(("Archivos de Excel", "*.xlsx"), ("todos los archivos", "*.*")))
print(root.filename)
padron = pd.read_excel(root.filename)
print(padron.head(5))

### duplicados